In [ ]:
LOAD_CONFIGS = {
    'data_path': "../data", # Directory that contains the expenses data
    'nubank_file_pattern': "nubank*.csv",
    'splitwise_groups': [
        'jacas-keter',
        '2021-mozi-e-eu',
        '2020-mozi-e-eu',
        '2019-mozi-e-eu',
        '2018-mozi-e-eu'
    ],
    'manual_file_pattern': "Extrato outras contas*Despesas*.csv",
    'incomes_file_pattern': "Extrato outras contas*Rendimentos*.csv",
    'person_who_pays': "Lucas Alencar", # Person name that the expenses will be extracted from Splitwise csv
}

# Year you were born
YEAR_OF_BIRTH = 1991

from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

from IPython.display import display

from src.load import investments
invest = investments.load(**LOAD_CONFIGS)

## Filtering parameters

Use these variable to filter portfolio by different timeframes and investments category.

### Date filters

Expected outputs from this cell are:

- `BASE_DATE`: date when the analysis is based on
- `BASE_PERIOD_START`: start date used for analysis in periods
- `BASE_PERIOD_END`: end date used for analysis in periods

For dates use `years_ago(base_date, years)` or `months_ago(base_date, months)` to filter the periods.

In [ ]:
from datetime import date
from src.date_helpers import years_ago, months_ago, beginning_of_month

BASE_DATE = months_ago(beginning_of_month(date.today()), 1)
BASE_DATE = date(2020, 12, 1)

BASE_PERIOD_START = years_ago(BASE_DATE, 1)
BASE_PERIOD_END = BASE_DATE

### Category filters

Expected outsputs from this cell are:

- `TITLES`: list of investments titles to be considered in the analysis

In [ ]:
from src import filters
types = []

#### Cripto types

In [ ]:
cripto_types = ['Criptomoedas', 'Bitcoin', 'Ether', 'DAI']
types += cripto_types

#### Variable income types

In [ ]:
variable_income_types = ['Fundos Ações', 
                         'Fundos Multimercado',
                         'Ações',
                         'Fundos Imobiliários',
                         'ETF',
                         'BDR']
types += variable_income_types

#### Fixed income types

In [ ]:
fixed_income_types = ['Renda Fixa', 
                      'Tesouro Direto',
                      'CDB',
                      'LCI',
                      'LCA',
                      'RDB',
                      'Tesouro SELIC',
                      'Tesouro IPCA',
                      'Fundos Renda Fixa']
types += fixed_income_types

In [ ]:
TITLES = filters.investment.currently_invested_titles_by_type(invest, types)

## Benchmark metrics

Use these metrics to understand how your portfolio is performing compared to inflation and stock market.

In [ ]:
from src.load import central_bank
current_ipca = central_bank.ipca_for_month(BASE_DATE)
current_ipca = current_ipca * 100 if current_ipca else None
print("IPCA:", current_ipca, "%")

# TODO Add monthlty IBOVESPA 

In [ ]:
from src.investments import monthly as monthly_investments

print(">>> Month of analysis:", BASE_DATE.month, BASE_DATE.year)

RETURN_FOR_MONTH_GOAL = 0.005
RETURN_WITH_INFLATION_GOAL = 0.0045

# Table with returns for the current month
summary_invest = monthly_investments.summary(invest, BASE_DATE).sort_values('Return for month (%)', ascending=False)

display(monthly_investments.style_summary(summary_invest, RETURN_FOR_MONTH_GOAL, RETURN_WITH_INFLATION_GOAL))